In [16]:
## 分類をインデックス化してDataFrameに追加する共通関数(秋山さん作成分)
def classify_and_add_indexed_df(df, org_key, new_key):
    class_sets = set()
    for class_ in df[org_key]:
        class_sets.add(class_)

    sorted_classes = sorted(list(class_sets))
    class_indices = dict((c, i) for i, c in enumerate(sorted_classes)) 
    print(class_indices)

    classes = []
    for i, class_ in enumerate(reviews[org_key]):
        classes.append(class_indices[class_])
    indexed_df[new_key] = classes

In [20]:
## メイン関数
import sys
import codecs
import pandas as pd
import matplotlib.pyplot as plt

## ファイルの読込み
shops = pd.read_csv('shops.txt', delimiter='\t')
reviews = pd.read_csv('reviews.txt', delimiter='\t')

## インデックスの作成
indexed_df = pd.DataFrame()
classify_and_add_indexed_df(reviews, '分類', 'category')
classify_and_add_indexed_df(reviews, 'スープ', 'soup')



# shops = pd.read_csv('as.txt', delimiter='\t')
# reviews = pd.read_csv('ar.txt', delimiter='\t')

#print(shops.columns[1])
#print(shops['店名'])

# print(reviews['コメント'])

{'塩': 3, '味噌': 2, '担々麺': 5, 'その他': 0, '豚骨醤油': 8, 'カレー': 1, '煮干し': 6, '鶏白湯': 11, '醤油': 10, '豚骨': 7, '豚骨魚介': 9, '塩豚骨': 4}
0          9
1          7
2         10
3         10
4         10
5          2
6         10
7          7
8          2
9          2
10        10
11        10
12         3
13        10
14        10
15         1
16         0
17         8
18         3
19         2
20         5
21         3
22        10
23         9
24         8
25         7
26         7
27         3
28         7
29         2
          ..
108275    10
108276     2
108277     0
108278     2
108279    10
108280    10
108281    11
108282    10
108283    10
108284    10
108285     9
108286     2
108287    10
108288    10
108289    10
108290     2
108291     8
108292     2
108293    10
108294    10
108295     4
108296     9
108297     7
108298     3
108299    10
108300    10
108301     0
108302     8
108303     4
108304    10
Name: soup, Length: 108305, dtype: int64
